# Packages and functions

In [1]:
!pip install unidecode

import numpy as np
import random
from random import randint
from datetime import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score
import pandas as pd
import numpy as np
from time import time
from gensim.models import Word2Vec
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from random import choice
from scipy.sparse import identity, diags
from unidecode import unidecode
from urllib.request import urlopen
import gzip
import pickle
from gensim.parsing.preprocessing import remove_stopwords
import re
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from tqdm.notebook import tqdm
import requests
import io
from scipy import sparse
import matplotlib.pyplot as plt



device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")

zsh:1: command not found: pip


In [2]:
def save_subgraph_in_file(nbr_nodes, source_path='../input_data/edgelist.txt', destination_path='../input_data/small_edgelist.txt'):
    G = nx.read_edgelist(source_path, delimiter=',', create_using=nx.Graph(), nodetype=int)
    G = G.subgraph(range(nbr_nodes))
    nx.write_edgelist(G, path=destination_path, delimiter=',')
    print(G.number_of_nodes(), 'nodes,', G.number_of_edges(), 'edges Graph extracted from', source_path[source_path.rfind('/')+1:])
    G = nx.read_edgelist(destination_path, delimiter=',', create_using=nx.Graph(), nodetype=int)
    print(G.number_of_nodes(), 'nodes,', G.number_of_edges(), 'edges Graph saved in', destination_path[destination_path.rfind('/')+1:])
    print(max(G.nodes))
    return


def read_train_val_graph(path='../input_data/edgelist.txt', shuffle=True, val_ratio=0.1):
    #gets the data from the file on the distant server
    G = nx.read_edgelist(urlopen('https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/edgelist.txt'), delimiter=',', create_using=nx.Graph(), nodetype=int)
    nodes = list(G.nodes())
    edges = list(G.edges())
    n = G.number_of_nodes()
    m = G.number_of_edges()
    
    permutation = np.array(range(n))
    mapping_permutation = dict(zip(range(n), range(n)))
    if shuffle:
        # shuffle the order of the edges without changing the labels
        random.shuffle(edges)
        G = nx.Graph()
        G.add_edges_from(edges)
        permutation = np.random.permutation(n)
        print(type(permutation))
        # create a mapping from old nodes labels to new nodes labels
        mapping_permutation = dict(zip(range(n), permutation))

        # shuffle G node labels according to the permutation
        G = nx.relabel_nodes(G, mapping_permutation)    
        
        edges = list(G.edges())
        nodes = list(G.nodes())


    print('Number of nodes:', n, 'number of edges:', m,'in the Complete set')

    node_to_idx = dict()
    for i, node in enumerate(nodes):
        node_to_idx[node] = i

    val_edges = list()
    G_train = G.copy()

    for edge in edges:
        if random.random() < val_ratio and edge[0] < n and edge[1] < n:
            val_edges.append(edge)
            G_train.remove_edge(edge[0], edge[1]) # We remove the val edges from the graph G

   
    #for edge in val_edges:
        

    n = G_train.number_of_nodes()
    m = G_train.number_of_edges()
    train_edges = list(G_train.edges())

    print('Number of nodes:', n, 'number of edges:', m, 'in the Training set')
    print('len(nodes)', len(nodes))

    y_val = [1]*len(val_edges)

    n_val_edges = len(val_edges)
    
    print('Creating random val_edges...')
    for i in range(n_val_edges):
        n1 = nodes[randint(0, n-1)]
        n2 = nodes[randint(0, n-1)]
        (n1, n2) = (min(n1, n2), max(n1, n2))
        while n2 >= n: #or (n1, n2) in train_edges:
            if (n1, n2) in train_edges:
                print((n1, n2), 'in train_edges:')
            n1 = nodes[randint(0, n-1)]
            n2 = nodes[randint(0, n-1)]
            (n1, n2) = (min(n1, n2), max(n1, n2))
        val_edges.append((n1, n2))

    y_val.extend([0]*(n_val_edges))
    
    ### From Giannis /!\
    val_indices = np.zeros((2,len(val_edges)))
    for i,edge in enumerate(val_edges):
        val_indices[0,i] = node_to_idx[edge[0]]
        val_indices[1,i] = node_to_idx[edge[1]]
    
    print('Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects')
    print('Loaded from', path[path.rfind('/')+1:], 'and with a training validation split ratio =', val_ratio)
    
    
    
    return G, G_train, train_edges, val_edges, val_indices, y_val, nodes, node_to_idx, permutation, mapping_permutation

def random_walk(G, node, walk_length):
    walk = [node]
  
    for i in range(walk_length-1):
        neibor_nodes = list(G.neighbors(walk[-1]))
        if len(neibor_nodes) > 0:
            next_node = choice(neibor_nodes)
            walk.append(next_node)
    walk = [node for node in walk] # in case the nodes are in string format, we don't need to cast into string, but if the nodes are in numeric or integer, we need this line to cast into string
    return walk


def generate_walks(G, num_walks, walk_length):
  # Runs "num_walks" random walks from each node, and returns a list of all random walk
    t = time()
    print('Start generating walks....')
    walks = list()  
    for i in range(num_walks):
        for node in G.nodes():
            walk = random_walk(G, node, walk_length)
            walks.append(walk)
        #print('walks : ', walks)
    print('Random walks generated in in {}s!'.format(round(time()-t)))
    return walks

def apply_word2vec_on_features(features, nodes, vector_size=128, window=5, min_count=0, sg=1, workers=8):
    t = time()
    print('Start applying Word2Vec...')
    wv_model = Word2Vec(vector_size=vector_size, window=window, min_count=min_count, sg=sg, workers=workers)
    wv_model.build_vocab(features)
    wv_model.train(features, total_examples=wv_model.corpus_count, epochs=5) 
    print('Word2vec model trained on features in {} min!'.format(round((time()-t)/60)))
    features_np = []
    for node in nodes:
        features_np.append(wv_model.wv[node])

    features_np = np.array(features_np)
    print(features_np.shape, 'features numpy array created in {} min!'.format(round((time()-t)/60)))
    return features_np


import numpy as np
import scipy.sparse as sp

def normalize_adjacency(A):
    n = A.shape[0]
    A = A + identity(n)
    degs = A.dot(np.ones(n))
    inv_degs = np.power(degs, -1)
    D_inv = diags(inv_degs)
    A_hat = D_inv.dot(A)
    return A_hat


# a proposed adj normalization, but we will keep the original one in the function after
def normalize_adj(adj):
    """Normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()



def create_and_normalize_adjacency(G):
    adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix of the training graph
    #adj = normalize_adjacency(adj)
    adj = normalize_adj(adj)
    print('Created a normalized adjancency matrix of shape', adj.shape)
    indices = np.array(adj.nonzero()) # Gets the positions of non zeros of adj into indices
    print('Created indices', indices.shape, 'with the positions of non zeros in adj matrix')
    return adj, indices

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


def text_to_list(text):
    return unidecode(text).split(',')

def intersection(lst1, lst2): # a function that returns the number of common items of two lists and 1 or 0 if there are common. This function will be used in add_authors_to_pairs to add this features to the pairs.
    lst3 = [value for value in lst1 if value in lst2]
    is_common = 1 if len(lst3)>0 else 0
    return len(lst3)+1, is_common+1


def add_authors_to_pairs (pairs, authors):
    authors = pd.DataFrame(authors)
    try: 
        pairs = pairs.detach().cpu().numpy()
    except:
        pass
        

    pairs_df = pd.DataFrame(np.transpose(pairs)).rename(columns={0: "paper_1", 1: "paper_2"})
    pairs_df = pairs_df.merge(authors, left_on='paper_1', right_on='paper_permut', how='left').rename(columns={'authors': "authors_1"})
    pairs_df = pairs_df.merge(authors, left_on='paper_2', right_on='paper_permut', how='left').rename(columns={'authors': "authors_2"})
    pairs_df.drop(['paper_id_x', 'paper_id_y'], axis=1, inplace=True)

    pairs_df['nb_common_author'] = pairs_df.apply(lambda row: intersection(row['authors_1'], row['authors_2'])[0], axis=1)
    pairs_df['is_common_author'] = pairs_df.apply(lambda row: intersection(row['authors_1'], row['authors_2'])[1], axis=1)

    pairs_tensor = torch.LongTensor(np.transpose(pairs_df[["paper_1", "paper_2", 'is_common_author', 'nb_common_author']].values.tolist())).to(device)
    
    return pairs_tensor


In [3]:
def read_and_clean_abstracts (nodes, sample_length=-1, abstracts_path = 'https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/abstracts.txt'):
    t = time()
    abstracts = dict()
    abstracts_list = list()
    f = urlopen(abstracts_path)
    
    for i, line in tqdm(enumerate(f)):
        if i == sample_length:
            break
        if i in nodes:
            node, abstract = str(line).lower().split('|--|')
            abstract = remove_stopwords(abstract)
            #abstract = re.sub(r"[,.;@#?!&$()-]", " ", abstract)
            abstract = re.sub(r"[^a-zA-Z0-9\s]", "", abstract)
            #abstract = re.sub(r"\\", " ", abstract)
            abstract = remove_stopwords(abstract)

            for word in abstract.split()[:-1]:
                #abstract = abstract.replace(word, stemmer.stem(word))
                abstract = abstract.replace(word, lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(lemmatizer.lemmatize(word), pos='s'), pos='a'), pos='n'), pos='v'), pos='r'))
            
            node = re.sub("[^0-9]", "", node)
            if i != int(node):
                print('i and node not the same', i, node)
            abstracts[int(node)] = abstract
            abstracts_list.append(abstract)
        
    print('Text loaded and cleaned in {:.0f} min'.format((time()-t)/60))
    return abstracts

def doc_counter (documents, word): #a function that return the number of documents containing a word
    counter = 0
    for i in documents:
        if word in documents[i]:
            counter += 1
    return counter



In [4]:
class Vocabulary:

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.word_occurrence = {}
        self.word2node = {}
        self.words_list = []
        self.sentences_list = []
        self.sentences_list_words = []
        self.num_words = 0
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word, node):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.words_list.append(word)
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
            self.word_occurrence[word] = 1
            self.word2node[word] = [node]
        else:
            # Word exists; increase word count
            self.word2count[word] += 1
            self.word_occurrence[word] += 1
            if node not in self.word2node[word]:
                self.word2node[word].append(node)
            # self.num_words += 1
            
    def add_sentence(self, sentence, node):
        sentence_len = 0
        for word in sentence.split()[:-1]:
            sentence_len += 1
            self.add_word(word, node)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1
        self.sentences_list.append(sentence)
        self.sentences_list_words.append(sentence.split()[:-1])

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

    def words(self):
        return self.words_list



In [5]:
def prepare_data_to_train (features, authors, adj, auth_matrix, indices, val_indices, y_val):
    
    print('Preparing the data for training...')
    
    t = time()
    
    y_val = torch.LongTensor(y_val).to(device)

    # Create class labels
    y = np.zeros(2*indices.shape[1])
    y[:indices.shape[1]] = 1 # Concatenated ones for edges indices and later in the model we add zeros for random indices.
    y = torch.LongTensor(y).to(device)
    
    features = torch.FloatTensor(features).to(device)
    
    indices = torch.LongTensor(indices).to(device)
    adj = sparse_mx_to_torch_sparse_tensor(adj).to(device)
    auth_matrix = sparse_mx_to_torch_sparse_tensor(auth_matrix).to(device)
    #tfidf_matrix = sparse_mx_to_torch_sparse_tensor(tfidf_matrix).to(device)
    
    # the function add_authors_to_pairs converts into torch tensors and sends to Device    
    val_indices = add_authors_to_pairs(val_indices, authors) #we add the authors to val_pairs
    indices = add_authors_to_pairs(indices, authors) #we add the authors to indices    
    rand_indices = np.random.randint(0, features.shape[0], (indices.shape[0],indices.shape[1]))# We take random indices each time we run an epoch
    rand_indices = add_authors_to_pairs(rand_indices, authors)

    pairs = torch.cat((indices, rand_indices), dim=1) # Concatenate the edges indices and random indices. 
    del(authors, indices, rand_indices)
    
    print('Data converted into torch tensors and authors added to indices in {:.0f} min'.format((time()-t)/60))

    return features, adj, auth_matrix, pairs, y, val_indices, y_val 

In [6]:
def map_features_with_permutation(features, permutation):
    new_features = np.zeros((len(features), len(features[0])))
    for i in range(len(features)):
        new_features[i] = features[permutation[i]]
    return new_features

# Load graph and authors data from sources

In [7]:
t = time()
shuffle = False
G, G_train, train_edges, val_edges, val_indices, y_val, nodes, node_to_idx, permutation, mapping_permutation = read_train_val_graph(val_ratio=0.1, shuffle=shuffle)

print('graph loaded and seperated, val indices generated and node to index mapping returned in {:.0f} s'.format(time()-t))

Number of nodes: 138499 number of edges: 1091955 in the Complete set
Number of nodes: 138499 number of edges: 982375 in the Training set
len(nodes) 138499
Creating random val_edges...
Returned G_train, train_edges, val_edges, y_val, nodes and node_to_idx objects
Loaded from edgelist.txt and with a training validation split ratio = 0.1
graph loaded and seperated, val indices generated and node to index mapping returned in 10 s


In [8]:
adj, indices = create_and_normalize_adjacency(G_train)


/var/folders/pd/glcnl2497w5b6xn3p94tnwlr0000gn/T/ipykernel_51428/1170815680.py:163: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G) # Obtains the adjacency matrix of the training graph


Created a normalized adjancency matrix of shape (138499, 138499)
Created indices (2, 1964750) with the positions of non zeros in adj matrix


/var/folders/pd/glcnl2497w5b6xn3p94tnwlr0000gn/T/ipykernel_51428/1170815680.py:155: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()


In [9]:
authors = pd.read_csv(urlopen('https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/authors.txt'), sep = '|', header=None)
authors = authors.rename(columns={0: "paper_id", 2: "authors"})
authors['authors'] = authors['authors'].apply(text_to_list)
authors = authors[["paper_id", "authors"]]
authors = authors[authors['paper_id'] <= max(G.nodes())]
authors['paper_permut'] = permutation[authors['paper_id']]
authors.head()

,paper_id,authors,paper_permut
0,0,"[James H. Niblock, Jian-Xun Peng, Karen R. McM...",0
1,1,"[Jian-Xun Peng, Kang Li, De-Shuang Huang]",1
2,2,[J. Heikkila],2
3,3,"[L. Teslic, B. Hartmann, O. Nelles, I. Skrjanc]",3
4,4,"[Long Zhang, Kang Li, Er-Wei Bai, George W. Ir...",4


In [10]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix


# get the unique list of authors
authors_lst_ppr = list(set([a for authors_list in tqdm(authors['authors']) for a in authors_list]))

# create a mapping of author to index
author_to_index = {author: i for i, author in tqdm(enumerate(authors_lst_ppr))}

# create an empty sparse matrix
nrows = len(authors)
ncols = len(authors_lst_ppr)
data = np.ones(nrows)
row_ind = np.arange(nrows)
col_ind = np.zeros(nrows)

# fill in the sparse matrix with 1s where authors appear
for i, authors_list in tqdm(enumerate(authors['authors'])):
    for author in authors_list:
        col_ind[i] = author_to_index[author]
        row_ind[i] = i
        data[i] = 1
auth_matrix = csr_matrix((data, (row_ind, col_ind)), shape=(nrows, ncols))

# print the resulting sparse matrix
print(auth_matrix.shape)


  0%|          | 0/138499 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

(138499, 147950)


In [11]:
# get the indices of non-zero elements
row_idx, col_idx = auth_matrix.nonzero()

# display the first non-zero element
print([row_idx[0], col_idx[0]])

[0, 21089]


In [12]:
auth_matrix.shape

(138499, 147950)

# Read processed data

In [13]:
t = time()
from io import BytesIO

walks_url = 'https://storage.googleapis.com/link_prediction_processed_data/walks_wv.npy'
with urlopen(walks_url) as url:
    data = url.read()

# Create a seekable file-like object from the data
fileobj = BytesIO(data)

# Load the data from the file object
walks_wv = np.load(fileobj)
print('wv walks loaded from GCP in {:.0f} sec'.format(time()-t))
walks_wv.shape

wv walks loaded from GCP in 0 sec


(138499, 64)

In [14]:
# Load the BART embedding torch tensor

# url = "https://storage.googleapis.com/link_prediction_processed_data/bart_embeddings.pt"
# response = requests.get(url)

# with open("bart_embeddings.pt", "wb") as f:
#     f.write(response.content)

# abstracts_bart_embeddings = torch.load('bart_embeddings.pt')


In [15]:
# # Load the wv300 mean embedding
# url = 'https://storage.googleapis.com/link_prediction_processed_data/embedded_abstracts_local_wv300.npy'

# response = requests.get(url)

# with open('embedded_abstracts_local_wv300.npy', 'wb') as f:
#     f.write(response.content)

# # Load the numpy array from the saved file
# local_wv300_abstracts = np.load('embedded_abstracts_local_wv300.npy')

# local_wv300_abstracts.shape


In [16]:
# # Load the goog300 mean embedding
# url = 'https://storage.googleapis.com/link_prediction_processed_data/embedded_abstracts_goog300.npy'

# response = requests.get(url)

# with open('embedded_abstracts_goog300.npy', 'wb') as f:
#     f.write(response.content)

# # Load the numpy array from the saved file
# goog300_abstracts = np.load('embedded_abstracts_goog300.npy')

# goog300_abstracts.shape


In [17]:
# # Load TF-IDF matrix

# url = "https://storage.googleapis.com/link_prediction_processed_data/tfidf_matrix.npz"

# response = requests.get(url)

# with open("tfidf_matrix.npz", "wb") as f:
#     f.write(response.content)

# tfidf_matrix = sparse.load_npz("tfidf_matrix.npz")

# tfidf_matrix.shape

In [18]:
# # Load words 192 embedding
# import gzip
# import pickle

# with gzip.open('embedded_abstracts_dict_192array.pkl.gz', 'rb') as f:
#     words_embedding_192 = pickle.load(f)

# len(words_embedding_192)

In [19]:
# len(words_embedding_192)

In [20]:
# from tqdm.notebook import tqdm
# import numpy as np

# words_embedding_192_trunc128 = dict ()
# for i in tqdm(range(len(words_embedding_192))):
#     if len(words_embedding_192[i])>0:
#         arr = np.zeros((128, 192))
#         vec = words_embedding_192[i][:128, :]
#         arr[:vec.shape[0],:] = vec
#         words_embedding_192_trunc128[i] = torch.tensor(arr).to(device)
#     else:
#         words_embedding_192_trunc128[i] = []


In [21]:
# tensor_list = []
# i = 0
# for i in tqdm(range(len(words_embedding_192_trunc128))):
#     if len(words_embedding_192_trunc128[i])>0:
#         tensor_list.append(words_embedding_192_trunc128[i])
#     else:
#         tensor_list.append(torch.zeros((128, 192)))


In [22]:
# del(words_embedding_192_trunc128)

In [23]:
# import gzip

# filename = 'words_embedding_192_trunc128.pkl.gz'

# # open the file in binary mode and write the dictionary to it, compressing the data with gzip
# with gzip.open(filename, 'wb') as f:
#     pickle.dump(words_embedding_192_trunc128, f)

In [24]:
# tensor_list_float = []
# for i, tensor in tqdm(enumerate(tensor_list)):
#     tensor_list_float.append(tensor_list[i].float())
# del(tensor_list)

In [25]:
# # Load truncated padded 128 words embeddings from GCP
# url = 'https://storage.googleapis.com/link_prediction_processed_data/words_embedding_192_trunc128.pkl.gz'


# # download the file from the URL
# response = requests.get(url)

# # save the file to disk
# with open(filename, 'wb') as f:
#     f.write(response.content)

# # load the data from the file
# with gzip.open(filename, 'rb') as f:
#     words_embedding_192_trunc128 = pickle.load(f)

In [ ]:
# Load truncated 128 words embeddings from URL

url = 'https://storage.googleapis.com/link_prediction_processed_data/words_embedding_192_trunc128.pkl.gz'


print('Loading file from URL...')
# open the file from URL in binary mode and read the dictionary from it, decompressing the data with gzip
with urllib.request.urlopen(url) as f:
    with gzip.open(f, 'rb') as f2:
        data = pickle.load(f2)

print('File loaded.')

In [26]:
# Load truncated padded 128 words embeddings from file

t = time()

filename = 'words_embedding_192_trunc128.pkl.gz'

print('Loading padded truncated embeddings...')
# open the file in binary mode and read the dictionary from it, decompressing the data with gzip
with gzip.open(filename, 'rb') as f:
    words_embedding_192_trunc128 = pickle.load(f)

print('padded truncated embeddings loaded in {:.0f} min'.format((time()-t)/60))

Loading padded truncated embeddings...
padded truncated embeddings loaded in 4 min


In [27]:
words_embedding_192_trunc128[0].shape

torch.Size([128, 192])

In [28]:
list_tensors = []
i = 0
for i in tqdm(range(len(words_embedding_192_trunc128))):
    if len(words_embedding_192_trunc128[i])>0:
        list_tensors.append(words_embedding_192_trunc128[i])
    else:
        list_tensors.append(torch.zeros((128, 192)))

  0%|          | 0/138499 [00:00<?, ?it/s]

In [29]:
del(words_embedding_192_trunc128)

tensor([[ 0.1693, -0.5117, -0.1612,  ...,  0.3282,  0.2673, -0.2871],
        [ 0.3021, -0.0629,  0.1254,  ...,  0.3228,  0.4747, -0.2569],
        [ 0.1363, -0.1517, -0.0926,  ...,  0.1736,  0.0540,  0.4056],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)

# Models

In [32]:
class GNN(nn.Module):
    def __init__(self, n_auth, n_feat, n_hidden, n_class, sub_class, dropout):
        super(GNN, self).__init__()
        self.auth_emb = nn.Linear(n_auth, n_hidden)
        self.fc1 = nn.Linear(n_feat, n_hidden)        
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(4*n_hidden, n_hidden)
        self.fc4 = nn.Linear(n_hidden, sub_class)
        self.fc5 = nn.Linear(sub_class, n_class)        
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()


       
        

    def forward(self, x_in, auth, adj, pairs):

        y = self.auth_emb(auth)
        y = self.relu(y)
        y = self.dropout(y)
        del(auth)

        #x_in = torch.cat((x_in, y), dim=1)
        
        h1 = self.fc1(x_in)
        z1 = self.relu(torch.spmm(adj, h1))
        z1 = self.dropout(z1)
        del(x_in, h1)

        h2 = self.fc2(z1)
        z2 = self.relu(torch.spmm(adj, h2))
        z2 = self.dropout(z2)
        del(h2, z1, adj)
        
        x = torch.cat((z2[pairs[0]] , y[pairs[0]], z2[pairs[1]] , y[pairs[1]]), dim=1)
        del(z2)
        # multiply by the authors flag
        # x = (pairs[2][:, None])*x       
        
        #x = torch.cat((x, y), dim=1)        
        del(pairs)

        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        
        x = self.relu(self.fc4(x))
        x = self.dropout(x)
        
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)




In [ ]:
import torch
import torch.nn as nn

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, output_dim):
        super().__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Convolutional layers
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1,
                      out_channels=num_filters,
                      kernel_size=(fs, embedding_dim))
            for fs in filter_sizes
        ])

        # Fully connected layer
        self.fc = nn.Linear(len(filter_sizes) * num_filters, output_dim)

        # Sigmoid activation function
        self.sigmoid = nn.Sigmoid()

    def forward(self, text):
        # text shape: [batch_size, max_sent_len]

        # Embedding layer
        embedded = self.embedding(text) # shape: [batch_size, max_sent_len, embedding_dim]

        # Add a channel dimension for convolutional layers
        embedded = embedded.unsqueeze(1) # shape: [batch_size, 1, max_sent_len, embedding_dim]

        # Convolutional layers
        conved = [nn.functional.relu(conv(embedded)).squeeze(3) for conv in self.convs] # shape: [batch_size, num_filters, max_sent_len - filter_size + 1]

        # Max pooling
        pooled = [nn.functional.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved] # shape: [batch_size, num_filters]

        # Concatenate pooled outputs from different filter sizes
        cat = torch.cat(pooled, dim=1) # shape: [batch_size, num_filters * len(filter_sizes)]

        # Fully connected layer
        output = self.fc(cat) # shape: [batch_size, output_dim]

        # Sigmoid activation
        output = self.sigmoid(output) # shape: [batch_size, output_dim]

        return output


In [124]:
model_CNN = CNN()

sampling_size = 20000

train_CNN(model_CNN, 0.01, shuffled_list_tensors[: sampling_size], shuffled_pairs_torch, shuffled_y_torch, shuffled_val_indices, shuffled_y_val_torch, 50, 2000, run_number=4, window = 10)


Initializing the optimizer with learning rate: 0.01
Start training...
Epoch: 001 loss_train: 3.1571 loss_val: 0.6925 acc_train: 0.5009 acc_val: 0.5177 time: 10 s total_time: 0 min
Epoch: 006 loss_train: 6.6591 loss_val: 4.1714 acc_train: 2.9600 acc_val: 3.0355 time: 11 s total_time: 1 min


KeyboardInterrupt: 

In [123]:

class CNN(nn.Module):
    def __init__(self, dtype=torch.float32):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=192, out_channels=64, kernel_size=8, stride=4, padding=3)
        #self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=8, stride=4, padding=3)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.fc1 = nn.Linear(2112, 128)
        self.fc2 = nn.Linear(128*2, 64)
        self.fc5 = nn.Linear(64, 2)
        self.dropout = nn.Dropout(0.2)
        self.dtype = dtype
    

    
    def forward(self, x1, x2):
        x1 = x1.permute(0, 2, 1)
        x2 = x2.permute(0, 2, 1)
        
        x1 = self.conv1(x1.to(self.dtype))
        x2 = self.conv1(x2.to(self.dtype))

        x1 = self.sigmoid(x1)
        x2 = self.sigmoid(x2)

        x1 = self.pool(x1)
        x2 = self.pool(x2)
#         print(x1.shape)

#         x1 = self.conv2(x1)
#         x2 = self.conv2(x2)
#         print(x1.shape)

#         x1 = self.relu(x1)
#         x2 = self.relu(x2)

#         x1 = self.pool(x1)
#         x2 = self.pool(x2)
#         print(x1.shape)

        x1 = x1.view(x1.shape[0], -1)
        x2 = x2.view(x2.shape[0], -1)

        x1 = self.fc1(x1)
        x2 = self.fc1(x2)

        x = torch.cat((x1, x2), dim=1)
        del(x2)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc5(x)

        return F.log_softmax(x, dim=1), x1



# Training functions

In [33]:
class EarlyStopping:
    def __init__(self, model, patience, delta, path='checkpoint.pt'):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.model = model
        self.val_loss_min = np.Inf
        
    def __call__(self, val_loss, path='checkpoint.pt'):
        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(path)
        elif score > self.best_score + 0:
            self.counter += 1
            #print(self.counter)
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(path)
            self.counter = 0

    def save_checkpoint(self, path):
        torch.save(self.model.state_dict(), path)

    def load_checkpoint(self, path):
        self.model.load_state_dict(torch.load(path))

In [94]:
def train_CNN(model, learning_rate, x, pairs, y, val_indices, y_val, epochs, batch_size, run_number, window = 10):
    # Train model
    start_time = time()
    
    print('Initializing the optimizer with learning rate:', learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #optimizer with halving learning rate in training

    today = datetime.today().strftime('%Y-%m-%d-%H:%M')
    list_loss_val = []
    list_loss_train = []
    list
    num_batches = len(x) // batch_size
    if len(x) % batch_size != 0:
        num_batches += 1
    
    
    halving_lr = 0 # counter of the number of halving lr
    patience = 16
    early_stopping = EarlyStopping(model, patience=patience, delta=0.1, path='checkpoint.pt')
    print('Start training...')

    total_loss = 0
    total_acc = 0
    total_loss_val = 0
    total_acc_val = 0
    num_pairs = pairs.shape[1]

    for epoch in range(epochs):
        # print(f'epoch {epoch}')
        t = time()
        optimizer.zero_grad()
        
        model.train()
        for i in (range(0, len(x), batch_size)):
            x_batch = torch.stack(x[i : i+batch_size])
            lst = range(i , i+batch_size)
            
            mask1 = np.isin(pairs[0], lst)
            mask2 = np.isin(pairs[1], lst)
            

            
            pairs1 = pairs[0][mask1 & mask2].numpy() - i # we soustract the number of batch * batch_size
            pairs2 = pairs[1][mask1 & mask2].numpy() - i # because x_batch contains elements from 0 to batch_size
            
            y_batch = y[mask1 & mask2]
            x1_batch = x_batch[pairs1.astype('int')]
            x2_batch = x_batch[pairs2.astype('int')]
            
            output, _ = model_CNN(x1_batch, x2_batch) # we run the model that gives the output.
            output.to(device)
            loss_train = F.nll_loss(output, y_batch) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
            acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_batch.cpu().numpy())# just to show it in the out put message of the training
            loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
            optimizer.step() # Performs a single optimization step (parameter update).
            total_loss += loss_train.item()
            total_acc += acc_train
            
            
        model.eval()    
        for i in (range(0, len(x), batch_size)):
            x_batch = torch.stack(x[i : i+batch_size])
            lst = range(i , i+batch_size)
            
            mask_val1 = np.isin(val_indices[0], lst)
            mask_val2 = np.isin(val_indices[1], lst)
            
            val_indices_batch_1 = val_indices[0][mask_val1 & mask_val2].numpy() - i # already a numpy
            val_indices_batch_2 = val_indices[1][mask_val1 & mask_val2].numpy() - i
            y_val_batch = y_val[mask_val1 & mask_val2]
            
            x1_batch = x_batch[val_indices_batch_1.astype('int')]
            x2_batch = x_batch[val_indices_batch_2.astype('int')]
            
            output, _ = model(x1_batch, x2_batch)
            output.to(device)
            loss_val = F.nll_loss(output, y_val_batch)
            acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val_batch.cpu().numpy())
            total_loss_val += loss_val
            total_acc_val += acc_val            
            
            
        loss_train = total_loss / num_batches
        acc_train = total_acc / num_batches
        loss_val = total_loss_val / num_batches
        acc_val = total_acc_val / num_batches
        list_loss_val.append(loss_val)
        list_loss_train.append(loss_train)
        
        
        #val_indices_torch = torch.Tensor(val_indices).long()
        #val_indices_np = np.array(val_indices) #transform into a tensor
#         print(type(val_indices_np))
#         x1 = x[val_indices_np[0].astype(int)]
#         x2 = x[val_indices_np[1].astype(int)]
        
#         x1 = x[val_indices[0]]
#         x2 = x[val_indices[1]]

#         x1 = x[val_indices[0].astype('int')]
#         x2 = x[val_indices[1].astype('int')]
#         output, _ = model(x1, x2)
#         output.to(device)
#         #y_val = torch.LongTensor(y_val).to(device)
#         loss_val = F.nll_loss(output, y_val)
#         list_loss_val.append(loss_val.item())
#         list_loss_train.append(loss_train.item())
#         acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())
        


        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train),
                  'loss_val: {:.4f}'.format(loss_val),
                  'acc_train: {:.4f}'.format(acc_train),
                  'acc_val: {:.4f}'.format(acc_val),
                  'time: {} s'.format(int(round(time()) - round(t))),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
            
            if epoch % 50 == 0:
                model_path = "outputs/{}-model-{}epochs-{}.pt".format(today, epoch, run_number)
                torch.save(model.state_dict(), model_path)
            

        
        early_stopping(loss_val)

        if early_stopping.early_stop:
            halving_lr += 1
            if halving_lr > 4:
                break
            learning_rate = learning_rate/10
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            early_stopping = EarlyStopping(model, patience=patience, delta=0.01, path='checkpoint.pt')
            print('Deviding the learning rate by 10. New learning rate: {}'.format(learning_rate))



    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model







# for key, tensor in words_embedding_dict.items():
#     output_dict[key] = model(tensor.unsqueeze(0), pairs).squeeze()

# # Print the output shape of each tensor
# for key, tensor in output_dict.items():
#     print(key, tensor.shape)

In [68]:
features_torch, adj_torch, auth_torch, pairs_torch, y_torch, val_indices_torch, y_val_torch = prepare_data_to_train(walks_wv, authors, adj, auth_matrix, indices, val_indices, y_val)

#abstracts_bart_embeddings = abstracts_bart_embeddings.to(device)
#goog300_abstracts_torch = torch.FloatTensor(goog300_abstracts).to(device)
#local_wv300_abstracts_torch = torch.FloatTensor(local_wv300_abstracts).to(device)
#tfidf_matrix_torch = sparse_mx_to_torch_sparse_tensor(tfidf_matrix).to(device)

Preparing the data for training...
Data converted into torch tensors and authors added to indices in 2 min


In [85]:
# shuffle the list of tensors and get the mapping dictionary
mapping_dict = {}
perm = np.random.permutation(len(list_tensors))
shuffled_list_tensors = [list_tensors[i] for i in perm]
for i, p in enumerate(perm):
    mapping_dict[i] = p


shuffled_pairs_torch = torch.zeros(size=pairs_torch.shape)
shuffled_val_indices = torch.zeros(size=val_indices.shape)

perm_pairs = torch.randperm(shuffled_pairs_torch.shape[1])
perm_val_indices = torch.randperm(shuffled_val_indices.shape[1])


pairs_dict = dict()

# Update pairs_torch tensor with the new indices
for i in tqdm(range(len(pairs_torch[0]))):
    old_idx1 = int(pairs_torch[0][i])
    old_idx2 = int(pairs_torch[1][i])
    new_idx1 = mapping_dict[old_idx1]
    new_idx2 = mapping_dict[old_idx2]
    shuffled_pairs_torch[0][i] = new_idx1
    shuffled_pairs_torch[1][i] = new_idx2
shuffled_pairs_torch = torch.index_select(shuffled_pairs_torch, dim=1, index=perm_pairs)
shuffled_y_torch = torch.index_select(y_torch, dim=0, index=perm_pairs)
    
for i in tqdm(range(len(val_indices[0]))):
    old_idx1 = int(val_indices[0][i])
    old_idx2 = int(val_indices[1][i])
    new_idx1 = mapping_dict[old_idx1]
    new_idx2 = mapping_dict[old_idx2]
    shuffled_val_indices[0][i] = new_idx1
    shuffled_val_indices[1][i] = new_idx2
shuffled_val_indices = torch.index_select(shuffled_val_indices, dim=1, index=perm_val_indices)
shuffled_y_val_torch = torch.index_select(y_val_torch, dim=0, index=perm_val_indices)


  0%|          | 0/3929500 [00:00<?, ?it/s]

  0%|          | 0/219160 [00:00<?, ?it/s]

In [103]:
model_CNN = CNN()

sampling_size = 10000

train_CNN(model_CNN, 0.01, shuffled_list_tensors[: sampling_size], shuffled_pairs_torch, shuffled_y_torch, shuffled_val_indices, shuffled_y_val_torch, 50, 2000, run_number=4, window = 10)


Initializing the optimizer with learning rate: 0.01
Start training...
torch.Size([864, 128, 192])
torch.Size([864, 192, 128])
torch.Size([864, 64, 32])
torch.Size([864, 64, 8])
torch.Size([864, 32, 2])


RuntimeError: Given input size: (32x1x2). Calculated output size: (32x1x0). Output size is too small

In [60]:
model_CNN = CNN()

train_CNN(model_CNN, 0.01, list_tensors, pairs_torch, y_torch, val_indices, y_val_torch, 50, 2000, run_number=4, window = 10)


Initializing the optimizer with learning rate: 0.01
Start training...
epoch 0


  0%|          | 0/139 [00:00<?, ?it/s]

  0%|          | 0/139 [00:00<?, ?it/s]

Epoch: 001 loss_train: 3.0146 loss_val: 0.9280 acc_train: 0.7814 acc_val: 0.8846 time: 484 s total_time: 8 min
epoch 1


  0%|          | 0/139 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [37]:

x_val_1 = []
x_val_2 = []

for pair in tqdm(zip(val_indices_torch[0], val_indices_torch[1])):
    if random() < 0.1:
        x_val_1.append(list_tensors[int(pair[0])])
        x_val_2.append(list_tensors[int(pair[1])])
    
len(x_val_1), x_val_1[0].shape




0it [00:00, ?it/s]

(219160, torch.Size([128, 192]))

In [ ]:
x_val_torch_1 = torch.stack(x_val_1).to(device)
#x_val_torch_2

In [ ]:
model_CNN(x_val_1, x_val_2)

In [ ]:
train_CNN(model_CNN, 0.01, list_tensors, pairs_torch, y_torch, val_indices, y_val_torch, 50, 1000, run_number=4, window = 10)


In [ ]:
val_indices[0].astype('int')

In [ ]:
model_CNN((list_tensors[0]), (list_tensors[2]))

In [ ]:
model_CNN(torch.stack(list_tensors[0:1]), torch.stack(list_tensors[2:3]))

In [ ]:
m = 30
indices[0][m:m+40]

In [ ]:
list_tensors[2:4]

In [ ]:
lst = range(0, 4)
arr1 = indices[0]
arr2 = indices[1]

# check which elements belong to the list
mask1 = np.isin(arr1, lst)
mask2 = np.isin(arr2, lst)

# select the elements that satisfy the condition
result1 = arr1[mask1 & mask2]
result2 = arr2[mask1 & mask2]

print(result1)
print(result2)

In [ ]:
y_val[5:8]

In [ ]:
def train_model(model, learning_rate, auth, features, adj, pairs, y, val_indices, y_val, epochs, run_number, window = 10):
    # Train model
    start_time = time()
    
    print('Initializing the optimizer with learning rate:', learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #optimizer with halving learning rate in training
    try: os.mkdir('./outputs')
    except: pass

    today = datetime.today().strftime('%Y-%m-%d-%H:%M')
    list_loss_val = []
    list_loss_train = []
    list_epochs = []
    
    halving_lr = 0 # counter of the number of halving lr
    patience = 16
    early_stopping = EarlyStopping(model, patience=patience, delta=0.1, path='checkpoint.pt')
    print('Start training...')
    for epoch in range(epochs):
        t = time()
        optimizer.zero_grad()
        
        model.train()

        output = model(features, auth, adj, pairs).to(device) # we run the model that gives the output.
        loss_train = F.nll_loss(output, y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
        acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y.cpu().numpy())# just to show it in the out put message of the training
        loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
        optimizer.step() # Performs a single optimization step (parameter update).
        
        model.eval()
        output = model(features, auth, adj, val_indices).to(device)
        #y_val = torch.LongTensor(y_val).to(device)
        loss_val = F.nll_loss(output, y_val)
        list_loss_val.append(loss_val.item())
        list_loss_train.append(loss_train.item())
        list_epochs.append(epoch)
        acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())
        


        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {} s'.format(int(round(time()) - round(t))),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
            
            if epoch % 50 == 0:
                model_path = "outputs/{}-model-{}epochs-{}.pt".format(today, epoch, run_number)
                torch.save(model.state_dict(), model_path)
            

        
        early_stopping(loss_val)

        if early_stopping.early_stop:
            halving_lr += 1
            if halving_lr > 4:
                break
            learning_rate = learning_rate/10
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            early_stopping = EarlyStopping(model, patience=patience, delta=0.01, path='checkpoint.pt')
            print('Deviding the learning rate by 10. New learning rate: {}'.format(learning_rate))



    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model, list_loss_val, list_loss_train, list_epochs



# Train the model

In [ ]:
# shuffle features the same way as the Graph
walks_wv_shuffled = map_features_with_permutation(walks_wv, permutation)

In [ ]:
features_torch, adj_torch, auth_torch, pairs_torch, y_torch, val_indices_torch, y_val_torch = prepare_data_to_train(walks_wv, authors, adj, auth_matrix, indices, val_indices, y_val)

#abstracts_bart_embeddings = abstracts_bart_embeddings.to(device)
#goog300_abstracts_torch = torch.FloatTensor(goog300_abstracts).to(device)
#local_wv300_abstracts_torch = torch.FloatTensor(local_wv300_abstracts).to(device)
#tfidf_matrix_torch = sparse_mx_to_torch_sparse_tensor(tfidf_matrix).to(device)

In [ ]:
auth_torch.shape[1]

In [ ]:
n_hidden = 64
dropout_rate = 0.2
sub_class = 32
n_class = 2
text_embedding = 0 #tfidf_matrix_torch
n_auth = auth_torch.shape[1]
n_features = features_torch.shape[1]


# Creates the model
model = GNN(n_auth, n_features, n_hidden, n_class, sub_class, dropout_rate).to(device)


In [ ]:
torch.cuda.empty_cache()


epochs = 400
run_number = randint(0, 1000)
learning_rate = 0.01


trained_model, list_loss_val, list_loss_train, list_epochs = train_model(model, learning_rate, auth_torch, 
                            features_torch, adj_torch, pairs_torch, 
                            y_torch, val_indices_torch, y_val_torch, epochs, run_number)


In [ ]:
%matplotlib inline


y1 = pd.Series(list_loss_val[:400])
y2 = pd.Series(list_loss_train[:400])
x = pd.Series(list_epochs[:400])




plt.plot(x, y1, label='Validation Loss')
plt.plot(x, y2, label='Training Loss')

plt.title('Loss over Epochs, G not shuffled')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
pairs_torch.shape

In [ ]:
break

In [ ]:
torch.cuda.empty_cache()


epochs = 602
run_number = randint(0, 1000)
learning_rate = 0.01


trained_model = train_model(model, learning_rate, text_embedding, 
                            features_torch, adj_torch, pairs_torch, 
                            y_torch, val_indices_torch, y_val_torch, epochs, run_number)


In [ ]:
# before shuffling
torch.cuda.empty_cache()


epochs = 602
run_number = randint(0, 1000)
learning_rate = 0.01


trained_model = train_model(model, learning_rate, text_embedding, 
                            features_torch, adj_torch, pairs_torch, 
                            y_torch, val_indices_torch, y_val_torch, epochs, run_number)


In [ ]:
np.shape(pairs_torch), y_torch.shape, val_indices_torch.shape, y_val_torch.shape

In [ ]:
torch.cuda.empty_cache()


epochs = 602
run_number = randint(0, 1000)
learning_rate = 0.01


trained_model = train_model(model, learning_rate, text_embedding, 
                            features_torch, adj_torch, pairs_torch, 
                            y_torch, val_indices_torch, y_val_torch, epochs, run_number)


In [ ]:
break

In [ ]:
# with dense2 1024 of BART
torch.cuda.empty_cache()


epochs = 200
run_number = randint(0, 1000)
learning_rate = 0.005


trained_model = train_model(model, learning_rate, abstracts_bart_embeddings, features_torch, adj_torch, pairs_torch, y_torch, val_indices_torch, y_val_torch, epochs, run_number)


In [ ]:
break

In [ ]:
# Without dense 1000
torch.cuda.empty_cache()


epochs = 200
run_number = randint(0, 1000)


trained_model = train_model(model, 0.01, tfidf_matrix_torch, features_torch, adj_torch, pairs_torch, y_torch, val_indices_torch, y_val_torch, epochs, run_number)


# Generate test file

In [ ]:
from datetime import datetime
import re

test_path = 'https://www.lix.polytechnique.fr/~nikolentzos/files/aai/challenge/test.txt'
node_pairs = list()
f = urlopen(test_path)

for line in f:
    t = str(line).split(',')
    t[0] = int(re.sub("[^0-9]", "", t[0]))
    t[1] = int(re.sub("[^0-9]", "", t[1]))
    node_pairs.append((node_to_idx[int(t[0])], node_to_idx[int(t[1])]))

node_pairs = np.transpose(node_pairs)
node_pairs = add_authors_to_pairs(node_pairs, authors)
#node_pairs = torch.LongTensor(node_pairs).to(device)

adj_torch = sparse_mx_to_torch_sparse_tensor(adj).to(device)
features_torch = torch.FloatTensor(walks_wv).to(device)

test_output = model(features_torch, text_embedding, adj_torch, node_pairs)
y_pred = torch.exp(test_output)
y_pred = y_pred.detach().cpu().numpy()

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])

today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 1000)
model_nb = 4

pd.DataFrame(y_pred_true, columns=['predicted']).to_csv(
"{}-submission-{}-{}.csv".format(today, model_nb, random_nb), header=True, index=True, index_label='id'
)

In [ ]:
features.shape

# BART

In [ ]:
import torch
from transformers import BartTokenizer, BartModel

# Load the BART model and tokenizer
model = BartModel.from_pretrained('facebook/bart-large')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')



In [ ]:
abstracts_bart_embeddings = []

# Define a function to generate embeddings for text
def get_bart_embeddings(text, model):
    # Tokenize the input text
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    # Generate embeddings using the BART model
    with torch.no_grad():
        model_output = model(**encoded_input)
        embeddings = model_output.last_hidden_state.mean(dim=1).squeeze()

    return embeddings

In [ ]:
for i in tqdm(range(len(abstracts_bart_embeddings), len(voc.sentences_list))):
    abstract = voc.sentences_list[i]
    abstracts_bart_embeddings.append(get_bart_embeddings(abstract, model))


In [ ]:
saved_abstracts_bart_embeddings = abstracts_bart_embeddings

In [ ]:
len(saved_abstracts_bart_embeddings), len(abstracts_bart_embeddings), len(voc.sentences_list)

In [ ]:
abstracts_bart_embeddings = torch.stack(abstracts_bart_embeddings)
abstracts_bart_embeddings.shape

In [ ]:
abstracts_bart_embeddings.to(device)

In [ ]:
torch.save(abstracts_bart_embeddings, 'bart_embeddings.pt')


#Draft

In [ ]:
# Testing
model.eval()
node_pairs = np.array(np.transpose(node_pairs))
node_pairs = torch.LongTensor(node_pairs).to(device)

test_output = model(features, adj, node_pairs)
y_pred = torch.exp(test_output)
y_pred = y_pred.detach().cpu().numpy()

y_pred_true = list()
for element in y_pred:
    y_pred_true.append(element[1])
    

    
today = datetime.today().strftime('%Y-%m-%d')
random_nb = randint(0, 1000)

pd.DataFrame(y_pred_true, columns={'predicted'}).to_csv(
"{}-submission-{}-{}.csv".format(today, model_nb, random_nb), header=True, index=True, index_label='id'
)

In [ ]:
#### New script with batches

def early_stopping(loss_train, list_loss_train, loss_val, list_loss_val, 
                   tolerance=0.01, patience=15):
    list_loss_val = list(list_loss_val)[-patience:]
    list_loss_train = list(list_loss_train)[-patience:]
    if (len(list_loss_val) == patience and loss_val > (sum(list_loss_val)/len(list_loss_val)) and loss_train + tolerance < loss_val) or (len(list_loss_train) == patience and loss_train > (sum(list_loss_train)/len(list_loss_train))):
        #print('train: {:.5f} val: {:.5f} mean val: {:.5f}'.format(loss_train, loss_val, (sum(list_loss_val)/len(list_loss_val))))
        return True
    return False
    

    
def train_model(model, learning_rate, features, adj, indices_mc, y, val_indices, 
                y_val, epochs, batch_size, wv_walk_size, 
                tolerence = 0.01, patience = 15, run_number=randint(0, 1000)):
    # Train model
    start_time = time()
    print('Initializing the optimizer with learning rate:', learning_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) #optimizer with halving learning rate in training
    try: os.mkdir('./outputs')
    except: pass
    print('Preparing the data for training...')        

    today = datetime.today().strftime('%Y-%m-%d-%H:%M')
    list_loss_val = []
    list_loss_train = []

    
    halving_lr = 0 # counter of the number of halving lr
    print('Start training...')
    for epoch in range(epochs):
        t = time()

        # we create the rand indices corresponding to non edges (their y = 0)
        # we could apply a condition on epoch to run rand_indices (for speed purposes)
        rand_indices = np.random.randint(0, len(indices_mc), size=(indices_mc.shape[0], indices_mc.shape[1]))
        rand_indices = add_authors_to_pairs(rand_indices, authors)
        pairs = np.concatenate((indices_mc, rand_indices), axis=1)
        pairs = torch.LongTensor(pairs).to(device)

        permutation = torch.randperm(pairs.size()[1])
        
        # batches
        for i in range(0, pairs.size()[1], batch_size):
            optimizer.zero_grad()

            elts_indices = permutation[i:i+batch_size]
            batch_pairs = pairs[:, elts_indices]
            batch_y = y[elts_indices]

        
            model.train()

            output = model(features, adj, batch_pairs, wv_walk_size).to(device) # we run the model that gives the output.
            loss_train = F.nll_loss(output, batch_y) # we are using nll_loss as loss to optimize, we store it in loss_train. We compare to y which is stable and contains the tag ones and zeros.
            acc_train = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), batch_y.cpu().numpy())# just to show it in the out put message of the training
            loss_train.backward() # The back propagation ? --> Computes the gradient of current tensor w.r.t. graph leaves
            optimizer.step() # Performs a single optimization step (parameter update).
        
        model.eval()
        output = model(features, adj, val_indices, wv_walk_size).to(device)
        #y_val = torch.LongTensor(y_val).to(device)
        loss_val = F.nll_loss(output, y_val)
        list_loss_val.append(loss_val.item())
        list_loss_train.append(loss_train.item())
        acc_val = accuracy_score(torch.argmax(output, dim=1).detach().cpu().numpy(), y_val.cpu().numpy())
        
        if epoch % 5 == 0:
            print('Epoch: {:03d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.item()),
                  'loss_val: {:.4f}'.format(loss_val.item()),
                  'acc_train: {:.4f}'.format(acc_train.item()),
                  'acc_val: {:.4f}'.format(acc_val.item()),
                  'time: {} s'.format(int(round(time()) - round(t))),
                 'total_time: {} min'.format(round((time() - start_time)/60)))
        if epoch % 50 == 0:
            model_path = "outputs/{}-model-{}epochs-{}.pt".format(today, epoch, run_number)
            torch.save(model.state_dict(), model_path)
        
        if int(loss_val.item()) > 5:
            break
            
        early = early_stopping(loss_train.item(), list_loss_train, loss_val.item(), list_loss_val, patience=15)        
        if early:
            halving_lr += 1
            if halving_lr > 5:
                break
            list_loss_val=[]
            learning_rate = learning_rate/10
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            print('Deviding the learning rate by 2. New learning rate: {:.6f}'.format(learning_rate))



    print("Optimization Finished in {} min!".format(round((time() - start_time)/60)))
    return model



In [ ]:
epochs = 1000

trained_model = train_model(model, 0.01, features, authors, adj, indices, y, torch.tensor(val_indices).to(device), torch.tensor(y_val).to(device), epochs)
